### Applying Inferential Statistics to the Tempo Estimation Problem

Broadly, my intention is to test whether there is identifiable objective information in the audio files that corresponds to my subjective tempo assignment from listening to songs and tapping out the beat.  To do this, I look at songs where `librosa.beat.tempo()` (using its default parameters) identified a tempo that could not be reconciled with any of the ones I identified subjectively.  My null hypothesis will correspond to the idea that LibROSA's identified tempo is as good an objective candidate as the first one I identified subjectively.  My alternative hypothesis will correspond to the idea that objective information favors my subjective tempo.

Specifically, for each song, my test is as follows:
1. Cut off the beginning and end of the song.
2. Transform the mel spectrogram data to make the distribution look more Gaussian (using the transformations and parameters discussed in [eda.ipynb](eda.ipynb)).
3. Take the first K principal components.
4. For each component:
  - Sample each Nth observation, where N corresponds to the period associated with the hypothesized tempo.
  - Take the standard deviation. (The smaller the standard deviation, the more evidence there is of a repeated pattern at the associated periodicity.)
  - Repeat the two substeps above N-1 more times, with each repetition offset by one time unit (spectrogram hop) from the previous one.
  - Take the mean of these N standard deviations.
5. Do step 4 once for the "alternative hypothesis tempo" and once for the "null hypothesis tempo."
6. For each of the K components, calculate the difference between the two values calculated in step 5.
7. For each of the K components, note whether the difference is negative. (Negative differences favor the alternative hypothesis, because they imply more evidence of periodicity at the alternative hypothesis tempo.)
8. Since principal components are, by construction, independent (at least linearly), treat the result (negative vs. non-negative) for each component as an independent Bernoulli trial.  Under the null hypothesis, they are like coin tosses: for each one, the probability of a negative value is 0.5.
9. Perform a binomial test of the hypothesis that p=0.5, against the alternative that p>0.5, where p is the probability that any given component will show a negative difference.  The statistic is the probability of getting X or more "heads" in K tosses of a fair coin, or `1-binom_cdf(X-1, K, .5)`.

The choice of K is arbitrary: higher values of K introduce more noise by including components that may contain little residual information, but lower values give fewer "trials" to test, so it's not clear what value would maximize the power of the test.  As a heuristic, I'll take every component that individually explains more than 1% of the variance.

In [1]:
GROUND_TRUTH_PATH = 'songs20190129.csv'
SONG_BASEPATH = 'sound/'
MELSPEC_BASEPATH = 'melspec/'
HOPLEN = 256
GET_DATA = True

In [2]:
import librosa
import numpy as np
import pandas as pd
import scipy
import pickle
import gzip
import os
import random
import math

In [3]:
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib nbagg
from IPython.display import Audio
from sklearn.decomposition import PCA

In [4]:
song_list = ['Collide', 'SallyGoRoundTheRoses', 'WithoutYouMP']

In [5]:
if GET_DATA:
    suffix = '.mp3'
    paces = pd.read_csv(GROUND_TRUTH_PATH, usecols=range(4), index_col=0, header=None)
    paces.index.name = 'song'
    paces.columns = ['pace0', 'pace1', 'pace2']
    right_tempo = {}
    wrong_tempo = {}
    for song in song_list:
        songpath = SONG_BASEPATH + song + suffix
        right_tempo[song] = round(paces.loc[song,'pace0'])
        wrong_tempo[song] = round(librosa.beat.tempo(librosa.load(songpath)[0])[0])
    print(right_tempo)
    print(wrong_tempo)

{'SallyGoRoundTheRoses': 151.0, 'Collide': 94.0, 'WithoutYouMP': 159.0}
{'SallyGoRoundTheRoses': 99.0, 'Collide': 123.0, 'WithoutYouMP': 108.0}


In [6]:
if not GET_DATA:
    wrong_tempo = {  # LibROSA-identified tempos
        'Collide': 123, 
        'SallyGoRoundTheRoses': 99,
        'WithoutYouMP': 108
        }
    right_tempo = {  # Subjectively identified tempos
        'Collide': 94, 
        'SallyGoRoundTheRoses': 151,
        'WithoutYouMP': 159
        }

In [7]:
# Convert from tempo to periodicity
def hops_per_beat(beats_per_minute):
    samples_per_second = 22050
    seconds_per_minute = 60
    samples_per_hop = 256
    beats_per_second = beats_per_minute / seconds_per_minute
    hops_per_second = samples_per_second / samples_per_hop
    hops_per_beat = hops_per_second / beats_per_second
    return(hops_per_beat)

In [8]:
# Transformation of data to look Gaussian
# (Standardization is not necessary for this analysis since all data are the in same units)
def preproc(x):
    return(np.log(x+1e-5))

In [9]:
# Calculate differences between mean standard deviations for right vs. wrong tempos
def diffs(ncomp, right_period, wrong_period):
    right_beat_stds_pc = [[].copy() for j in range(ncomp)]
    wrong_beat_stds_pc = [[].copy() for j in range(ncomp)]
    for i in range(right_period):
        for j in range(ncomp):
            pcs = pc[j,i::right_period]
            p = right_beat_stds_pc[j].append(np.std(pcs))
    for i in range(wrong_period):
        for j in range(ncomp):
            pcs = pc[j,i::wrong_period]
            wrong_beat_stds_pc[j].append(np.std(pcs))
    return [np.mean(np.array(right_beat_stds_pc[j])) - \
     np.mean(np.array(wrong_beat_stds_pc[j])) for j in range(ncomp) ]

In [10]:
song = song_list[0]

In [11]:
pvals = []
for song in song_list:

    print('\n\n\nSong: ', song)

    # Get saved mel spectrogram
    melspec_path = MELSPEC_BASEPATH + song + '_melspec.pkl.gz'
    with gzip.open(melspec_path,'rb') as fp:
        ms = pickle.load(fp)  # Beginning and end of song are already removed

    print('\nShape of mel spectrogram: ', ms.shape)

    pca = PCA(n_components=40)
    pc = np.transpose(pca.fit_transform(np.transpose(preproc(ms))))
    print('Shape of principal components: ', pc.shape)

    print('\nExplained variance ratios:')
    print(pca.explained_variance_ratio_)

    min_explained = 0.01
    ncomp = (pca.explained_variance_ratio_ > min_explained).sum()
    print( '\nNumber of components to use:', ncomp)
    right_period = int(round(hops_per_beat(right_tempo[song])))
    print( 'Subjective periodicity:     ', right_period)
    wrong_period = int(round(hops_per_beat(wrong_tempo[song])))
    print( 'LibROSA periodicity:        ', wrong_period)

    diff = diffs(ncomp, right_period, wrong_period)
    print('\nDifferences of mean SDs:')
    print(diff)

    tosses = [d<0 for d in diff]
    print('\nCumulative mean of fraction that are negative:')
    print(np.array(tosses).cumsum()/(1+np.array(range(len(tosses)))))

    pval = 1 - scipy.stats.binom.cdf(sum(tosses)-1, len(tosses), .5)
    print('\nSignificance level:', pval)
    pvals.append(pval)




Song:  Collide

Shape of mel spectrogram:  (128, 16704)
Shape of principal components:  (40, 16704)

Explained variance ratios:
[0.28971815 0.06917114 0.06429128 0.04074661 0.03691182 0.03371659
 0.02850039 0.0256117  0.02327849 0.01711042 0.01668253 0.01479176
 0.01440992 0.01295199 0.01263691 0.01149125 0.01110762 0.01045921
 0.01004312 0.00948813 0.00888045 0.00842438 0.00809894 0.00774493
 0.00727744 0.0069554  0.00694053 0.00658439 0.00643772 0.00623526
 0.0060518  0.00565352 0.00537967 0.00518594 0.00505767 0.00488317
 0.00483063 0.00463455 0.00437611 0.00432162]

Number of components to use: 19
Subjective periodicity:      55
LibROSA periodicity:         42

Differences of mean SDs:
[-0.90404415, -0.04633379, -0.045452118, -0.05665493, -0.022636414, -0.00536108, -0.050516844, -0.016252995, -0.025046587, -0.025487661, -0.018435001, -0.026423454, -0.028068304, -0.022996664, -0.0070090294, -0.029582262, -0.008337975, -0.008699298, -0.005719781]

Cumulative mean of fraction that 

In [12]:
bonferroni = min(pvals)*len(pvals)
print('Corrected minimum significance level:', bonferroni)

Corrected minimum significance level: 5.7220458984375e-06


In summary, for *Collide* I conclusively reject the null hypothesis.  

For *Sally Go Round the Roses* there is no evidence for the alternative.  

For *Without You* I cannot reject the null, but, given the rejection for *Collide* and the negative differences for the first four components, the data seem consistent with the alternative hypothesis.